In [ ]:
%load_ext autoreload
%autoreload 2

import open3d as o3d
import numpy as np
import copy
from os import listdir
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
from winsound import Beep
import time
import glob

import alexutils as alx




In [ ]:
src = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\different_slams\mulls_results\5_binary\result\pose_b_lo_xxx_id.txt"
def get_transform_matrices_from_kitti(src):
    def line_to_transformation_matrix(line):
        l = line.replace("\n", "")
        l = (l + " 0 0 0 1").split(" ") 
        return np.reshape(np.array(l, dtype=float), (4,4))

    with open(src) as f:
        lines = f.readlines()
    return [line_to_transformation_matrix(line) for line in lines]

arr = get_transform_matrices_from_kitti(src)


In [ ]:
mat = np.loadtxt(r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Blender\110_gt.txt")
mat[0,3] = mat[0,3] - 0.5

In [ ]:
np.matmul(arr[1], mat)

In [ ]:
folder = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\different_slams\mulls_results\5_binary\result\single_transformations\\"
for idx, a in enumerate(arr):
    np.save(f"{folder}{idx}.npy", a)

In [ ]:
src = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\different_slams\mulls_results\5_binary\result\pose_b_lo_xxx_id.txt"
pcd_folder = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\different_slams\MULLS\demo_data\pcd\\"
pcd_target_folder = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\different_slams\MULLS\demo_data\pcd\transformed\\"

transform_matrices = get_transform_matrices_from_kitti(src)

filenames = listdir(pcd_folder)[:100]

for idx, file in enumerate(filenames):
    print(pcd_folder + file)
    point_cloud = o3d.io.read_point_cloud(pcd_folder + file)
    point_cloud.transform(transform_matrices[idx])
    o3d.io.write_point_cloud(pcd_target_folder + file, point_cloud, write_ascii = False)
    print(f"{idx}")

### Merge all vehicle frames

In [ ]:

src = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\different_slams\mulls_results\5_binary\result\pose_b_lo_xxx_id.txt"
pcd_folder = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\different_slams\MULLS\demo_data\pcd\\"
pcd_target_folder = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\different_slams\MULLS\demo_data\pcd\transformed\\"

transform_matrices = get_transform_matrices_from_kitti(src)
print(f"Number of transform matrices {len(transform_matrices)}")

filenames = [x.split("\\")[-1] for x in glob.glob(pcd_folder + "*.pcd")]
print(f"Number of pcd files: {len(filenames)}")
pcl = o3d.geometry.PointCloud() #Empty point cloud

for idx, file in enumerate(filenames):
    point_cloud = o3d.io.read_point_cloud(pcd_folder + file)
    if idx == 0:
        continue # skip drone data
    point_cloud.transform(transform_matrices[idx])
    pcl += point_cloud
    # print(str(idx))

print(f"Number of points with ground {len(pcl.points)}")
no_ground_pcd = alx.remove_ground(pcl, 0.5)
print(f"Number of points without ground {len(no_ground_pcd.points)}")
no_ground_pcd = no_ground_pcd.voxel_down_sample(voxel_size = 0.1)
print(f"Number of points with downsampling {len(no_ground_pcd.points)}")
alx.draw_point_clouds([no_ground_pcd])
o3d.io.write_point_cloud(pcd_target_folder + "merged_vehicle_frames_no_ground.pcd", no_ground_pcd, write_ascii = False)

In [ ]:
target = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\different_slams\MULLS\demo_data\pcd\transformed\00001.pcd"
source = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\different_slams\MULLS\demo_data\pcd\transformed\merged_vehicle_frames_no_ground.pcd"
source = o3d.io.read_point_cloud(source)
target = o3d.io.read_point_cloud(target)
alx.draw_point_clouds([target, source])

eval = alx.compute_icp_p2p(source, target)


alx.draw_point_clouds([target, alx.transform(source, eval.transformation)])
